# To do

- [X] write usable json files
- [X] allocate all files in folder
- [X] print multiple poi on map
- [ ] connect to open folder/files
- [ ] make it runable - GUI Win/Html....

# Info

# Sources

[Source 1](https://nbviewer.jupyter.org/github/gotoariel/folium-demo/blob/master/demo/demo.ipynb)

[Source 2](http://blog.schockwellenreiter.de/2020/05/2020051601.html)

In [66]:
# env info
!conda info


     active environment : geofiles
    active env location : /home/andres/miniconda3/envs/geofiles
            shell level : 2
       user config file : /home/andres/.condarc
 populated config files : 
          conda version : 4.8.4
    conda-build version : not installed
         python version : 3.7.6.final.0
       virtual packages : __cuda=10.2
                          __glibc=2.31
       base environment : /home/andres/miniconda3  (writable)
           channel URLs : https://repo.anaconda.com/pkgs/main/linux-64
                          https://repo.anaconda.com/pkgs/main/noarch
                          https://repo.anaconda.com/pkgs/r/linux-64
                          https://repo.anaconda.com/pkgs/r/noarch
          package cache : /home/andres/miniconda3/pkgs
                          /home/andres/.conda/pkgs
       envs directories : /home/andres/miniconda3/envs
                          /home/andres/.conda/envs
               platform : linux-64
             user-agent :

# Install libraries

In [5]:
conda install -c conda-forge folium
conda install -c conda-forge geocoder
pip install ediblepickle
conda install -c conda-forge matplotlib

Solving environment: done

## Package Plan ##

  environment location: /home/andres/miniconda3

  added / updated specs:
    - folium


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.8.4                |   py37hc8dfbb8_2         3.1 MB  conda-forge
    libblas-3.8.0              |      17_openblas          11 KB  conda-forge
    libcblas-3.8.0             |      17_openblas          11 KB  conda-forge
    libgfortran-ng-7.5.0       |      hdf63c60_16         1.3 MB  conda-forge
    liblapack-3.8.0            |      17_openblas          11 KB  conda-forge
    libopenblas-0.3.10         |pthreads_hb3c22a3_4         7.8 MB  conda-forge
    numpy-1.19.1               |   py37h7ea13bd_2         5.2 MB  conda-forge
    ------------------------------------------------------------
                                           Total:        17.3 MB

The following NEW packages will be INSTAL

# Import libraries

In [2]:
import json, string

#Folium uses the Leaflet API to allow users to write Python code to generate and manipulate interactive JavaScript maps. This also allows for drawing those maps in Jupyter notebooks
import folium
#import requests
import geocoder
#import pandas as pd
import matplotlib
import numpy as np
import pandas as pd
import os

from folium import Map, Marker#, GeoJson, LayerControl
#from ediblepickle import checkpoint


%matplotlib inline 

## Geocoder test

In [3]:
# get my locaton
my_loc = geocoder.ip('me')
print(my_loc.latlng)

[47.9959, 7.8522]


In [4]:
# get geocode via my ip
g = geocoder.ipinfo('93.218.39.223')
g.latlng

[47.9959, 7.8522]

# 1.0 Import data from dir

In [44]:
path = 'data//'

In [45]:
# def read dirs and create lists

file_dict = []
path_list = []

for root, dirs, files in os.walk(path): # searches in folders, subfolders and files
    for name in dirs:
        file_path = os.path.join(root, name)
        for txt_file in os.listdir(file_path):
            
            if txt_file == 'location.txt':
                file = os.path.join(file_path, txt_file) # get filepath for each file
                path_list.append(file)

                with open(file) as json_file: # collect json contend
                    data = json.load(json_file)
                    file_dict.append(data) # append json dict to file_dict


In [8]:
for keys, values in file_dict[0].items():
    #for i in values:
    print(values)

RÜB_59
Weinbergblick, 79395 Neuenburg am Rhein
Regenbecken
1998
47.784599, 7.543373


In [9]:
# build df with locations
df = pd.DataFrame.from_dict(file_dict)
df

,name,adress,desc,year_build,location
0,RÜB_59,"Weinbergblick, 79395 Neuenburg am Rhein",Regenbecken,1998,"47.784599, 7.543373"
1,Hof,"79395 Neuenburg am Rhein, Germany",Regenbecken,1900,"47.784599, 7.543373"
2,Kirche,"Kirchstrasse 20, 79395, Neuenburg, Germany",Regenbecken,1998,"47.767964, 7.555197"
3,RÜB_55,"St. Barbara Strasse 20, 79395, Neuenburg, Germany",Regenbecken,1998,"47.769831, 7.554888"


In [10]:
df['location'][0]

'47.784599, 7.543373'

In [11]:
#print(file_dict)

for i in file_dict:
    for keys, values in i.items():
        print(values)
        

RÜB_59
Weinbergblick, 79395 Neuenburg am Rhein
Regenbecken
1998
47.784599, 7.543373
Hof
79395 Neuenburg am Rhein, Germany
Regenbecken
1900
47.784599, 7.543373
Kirche
Kirchstrasse 20, 79395, Neuenburg, Germany
Regenbecken
1998
47.767964, 7.555197
RÜB_55
St. Barbara Strasse 20, 79395, Neuenburg, Germany
Regenbecken
1998
47.769831, 7.554888


In [12]:
location = df['location'][0]
print(type(location))
float(location.split(",")[1])

<class 'str'>


7.543373

In [191]:
g = geocoder.osm(location, method='reverse')
g

<[OK] Osm - Reverse [Im Malzacker, Steinenstadt, Landkreis Breisgau-Hochschwarzwald, Baden-Württemberg, 79395, Deutschland]>

In [32]:
latlng = df['location'][0].split(",")
lat = float(latlng[0])
lng = float(latlng[1])

In [30]:
print(type(latlng[0]))

<class 'str'>


## Testing data scrawler

In [89]:
# basik info
df.count()

name          4
adress        4
desc          4
year_build    4
location      4
dtype: int64

# Mapping

In [13]:
# iterate through df
for i in df.itertuples():
    print(i)
    break
#    for name, adress, desc, year_build, location in columns:
#        print(location)
#    print(locations)

Pandas(Index=0, name='RÜB_59', adress='Weinbergblick, 79395 Neuenburg am Rhein', desc='Regenbecken', year_build=1998, location='47.784599, 7.543373')


In [133]:
# work with NumPy Series, cause more efficient than Python dict
locations = pd.Series(file_dict)
locations[0]['location'].split(",")

['47.76983115', ' 7.5548874']

In [60]:
locations = pd.Series(file_dict)
location_base  = locations[0]['location'].split(",")
location_addr = locations[0]['adress']

locations_map = Map(location = location_base,
                    height= '100',
                    tiles = 'OpenStreetMap',
                    zoom_start = 8)
locations_map.add_child(
   Marker(location = locations[0]['location'].split(","),
                              popup = location_addr,
                              icon = folium.Icon(color = 'blue')))



In [ ]:
location

In [61]:
for i in pd.Series(file_dict):
    name = i['name']
    address = i['adress']
    description = i['desc']
    locations = i['location'].split(",")
    year_build = i['year_build']
   # print(locations[0])
   # print(location)
    folium.features.RegularPolygonMarker(location = [locations[0], locations[1]],
                                        tooltip = name,
                                        popup = "Name: " + name + "\nTyp: " + description + "\nAdresse: " + address,
                                        #zoom = 200,
                                        #number_of_sides = 3,
                                        #radius = 5,
                                        #weight = 1,
                                        #fill_opacity = 0.8
                                        ).add_to(locations_map)
    
locations_map

In [ ]:
# Saving maps

# user_map.save('current_map.html

# 2.0 reverse geocoding

## 2.1 write dict

In [46]:
g = geocoder.osm("47.76983115, 7.5548874", method='reverse')
g

<[OK] Osm - Reverse [Im Malzacker, Steinenstadt, Landkreis Breisgau-Hochschwarzwald, Baden-Württemberg, 79395, Deutschland]>

In [45]:
c = geocoder.osm([-25.3127, -57.6108], method='reverse')
c

<[OK] Osm - Reverse [Avenida Pedro Bruno Guggiari, Vista Alegre, Asunción, Región Oriental, 001311, Paraguay]>

In [46]:
results = geocoder.osm('St. Barbara Strasse 10, Steinenstadt, Germany')
results.json # show as json file, als .geojason, .wkt, .osm

{'accuracy': 0.42099999999999993,
 'address': '10, St. Barbara Straße, Steinenstadt, Landkreis Breisgau-Hochschwarzwald, Baden-Württemberg, 79395, Deutschland',
 'bbox': {'northeast': [47.7695802, 7.5548826],
  'southwest': [47.7694819, 7.5546943]},
 'confidence': 10,
 'country': 'Deutschland',
 'country_code': 'de',
 'county': 'Landkreis Breisgau-Hochschwarzwald',
 'housenumber': '10',
 'importance': 0.42099999999999993,
 'lat': 47.76953105,
 'lng': 7.55478845,
 'ok': True,
 'osm_id': 162443389,
 'osm_type': 'way',
 'place_id': 124173898,
 'place_rank': 30,
 'postal': '79395',
 'quality': 'yes',
 'raw': {'place_id': 124173898,
  'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
  'osm_type': 'way',
  'osm_id': 162443389,
  'boundingbox': ['47.7694819', '47.7695802', '7.5546943', '7.5548826'],
  'lat': '47.76953105',
  'lon': '7.55478845',
  'display_name': '10, St. Barbara Straße, Steinenstadt, Landkreis Breisgau-Hochschwarzwald, Baden-Württemberg, 7

In [116]:
#  as osmand results
results.osm

{'x': 7.55478845,
 'y': 47.76953105,
 'addr:housenumber': '10',
 'addr:street': 'St. Barbara Straße',
 'addr:state': 'Baden-Württemberg',
 'addr:country': 'Deutschland',
 'addr:postal': '79395'}

## 2.2 or read from files

In [54]:
results.json # display location json
results.latlng

[47.76953105, 7.55478845]

# 3.0 Add Href to open content

In [63]:
path_list

['data//datassss 2/location.txt',
 'data//TEST4a/location.txt',
 'data//test 6/location.txt',
 'data//Test 2/location.txt',
 'data//Test 1/location.txt',
 'data//Test3/location.txt']

In [72]:
for i in pd.Series(file_dict):
    name = i['name']
    address = i['adress']
    description = i['desc']
    locations = i['location'].split(",")
    year_build = i['year_build']
   # print(locations[0])
   # print(location)
    folium.features.RegularPolygonMarker(location = [locations[0], locations[1]],
                                        tooltip = name,
                                        popup = ('<a href=/home/andres/miniconda3/envs/geofiles/"data/datassss 2/location.txt">open dir</a>'), #"Name: " + name + "\nTyp: " + description + "\nAdresse: " + address,
                                        #zoom = 200,
                                        #number_of_sides = 3,
                                        #radius = 5,
                                        #weight = 1,
                                        #fill_opacity = 0.8
                                        ).add_to(locations_map)
    
locations_map